In [43]:
import numpy as np
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications import VGG16
from PIL import Image

In [33]:
def compile_train_plot_conf(network, tr_gen, val_gen):
    network.compile(loss='binary_crossentropy',
                    optimizer=optimizers.RMSprop(learning_rate=1e-4),
                    metrics=['acc'])
    history = model.fit(
        tr_gen,
        steps_per_epoch=50,
        epochs=30,
        validation_data=val_gen,
        validation_steps=50,
        workers=-1)
    print(history.history.keys())

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, 'yo', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [30]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

test_dir = 'chest_xray/test'
train_dir = 'chest_xray/train'
validation_dir = 'chest_xray/val'

img_dim = 256

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_dim, img_dim),
    batch_size=20,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_dim, img_dim),
    batch_size=20,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


# Basic Model

In [31]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3),
                        activation='relu',
                        input_shape=(img_dim, img_dim, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 30, 30, 128)     

In [32]:
compile_train_plot_conf(network=model, tr_gen=train_generator, val_gen=validation_generator)

Epoch 1/30
100/100 [==============================] - ETA: 0s - loss: 0.4007 - acc: 0.8096WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 50 batches). You may need to use the repeat() function when building your dataset.


100/100 [==============================] - 37s 364ms/step - loss: 0.4007 - acc: 0.8096 - val_loss: 0.4817 - val_acc: 0.8750
Epoch 2/30
100/100 [==============================] - 38s 377ms/step - loss: 0.1775 - acc: 0.9340
Epoch 3/30
100/100 [==============================] - 35s 353ms/step - loss: 0.1533 - acc: 0.9430
Epoch 4/30
 59/100 [================>.............] - ETA: 14s - loss: 0.1326 - acc: 0.9551

KeyboardInterrupt: 

# Basic Model + Augmentation

In [21]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.3,
    fill_mode='nearest')
# no Horizontal Flip :)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator_datagen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_dim, img_dim),
    batch_size=32,
    class_mode='binary')

validation_generator_datagen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_dim, img_dim),
    batch_size=32,
    class_mode='binary')

In [ ]:
compile_train_plot_conf(network=model, tr_gen=train_generator_datagen, val_gen=validation_generator_datagen)

# VGG16 + Augmentation (Transfer Learning)


In [24]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(img_dim, img_dim, 3))

conv_base.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

In [34]:
conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [35]:
compile_train_plot_conf(network=model, tr_gen=train_generator, val_gen=validation_generator)

Epoch 1/30
43/50 [========================>.....] - ETA: 11s - loss: 0.4333 - acc: 0.8302

KeyboardInterrupt: 

# Fine tuning


In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
compile_train_plot_conf(network=model, tr_gen=train_generator, val_gen=validation_generator)

# Testing on image not from dataset

In [61]:
img = Image.open("self_chest_xray.png").convert("RGB")
img = img.resize((img_dim, img_dim))
img_arr = np.array(img)
img_arr = img_arr.reshape((1,) + img_arr.shape)
model.predict(img_arr)

1/1 [==============================] - 0s 114ms/step


array([[1.]], dtype=float32)